In [1]:
using Pkg
# cd("D:\\Home\\Git\\Oceananigans.jl")
cd("C:\\Users\\Ali\\Documents\\Git\\Oceananigans.jl\\")
Pkg.activate(".");

In [2]:
using Statistics

using Oceananigans, Oceananigans.Operators

In [3]:
∇²

UndefVarError: UndefVarError: ∇² not defined

In [4]:
∇²!

UndefVarError: UndefVarError: ∇²! not defined

In [5]:
@inline incmod1(a, n) = a == n ? one(a) : a + 1
@inline decmod1(a, n) = a == 1 ? n : a - 1

function ∇²(f)
    Nx, Ny, Nz = size(f)
    ∇²f = zeros(Nx, Ny, Nz)
    for k in 2:(Nz-1), j in 1:Ny, i in 1:Nx
       ∇²f[i, j, k] = f[incmod1(i, Nx), j, k] + f[decmod1(i, Nx), j, k] + f[i, incmod1(j, Ny), k] + f[i, decmod1(j, Ny), k] + f[i, j, k+1] + f[i, j, k-1] - 6*f[i, j, k]
    end
    for j in 1:Ny, i in 1:Nx
        ∇²f[i, j,   1] = -(f[i, j,     1] - f[i, j,   2]) + f[incmod1(i, Nx), j,   1] + f[decmod1(i, Nx), j,   1] + f[i, incmod1(j, Ny),   1] + f[i, decmod1(j, Ny),   1] - 4*f[i, j,   1]
        ∇²f[i, j, end] =  (f[i, j, end-1] - f[i, j, end]) + f[incmod1(i, Nx), j, end] + f[decmod1(i, Nx), j, end] + f[i, incmod1(j, Ny), end] + f[i, decmod1(j, Ny), end] - 4*f[i, j, end]
    end
    ∇²f
end

∇² (generic function with 1 method)

In [43]:
@inline incmod1(a, n) = a == n ? one(a) : a + 1
@inline decmod1(a, n) = a == 1 ? n : a - 1

function ∇²!(g::RegularCartesianGrid, f::CellField, ∇²f::CellField)
    for k in 2:(g.Nz-1), j in 1:g.Ny, i in 1:g.Nx
       ∇²f.data[i, j, k] = (f.data[incmod1(i, g.Nx), j, k] - 2*f.data[i, j, k] + f.data[decmod1(i, g.Nx), j, k]) / g.Δx +
                           (f.data[i, incmod1(j, g.Ny), k] - 2*f.data[i, j, k] + f.data[i, decmod1(j, g.Ny), k]) / g.Δy +
                           (f.data[i, j, k+1]              - 2*f.data[i, j, k] + f.data[i, j, k-1])              / g.Δz
    end
    for j in 1:g.Ny, i in 1:g.Nx
        ∇²f.data[i, j,   1] = (f.data[i, j, 2] - f.data[i, j, 1]) / g.Δz +
                              (f.data[incmod1(i, g.Nx), j, 1] - 2*f.data[i, j, 1] + f.data[decmod1(i, g.Nx), j, 1]) / g.Δx +
                              (f.data[i, incmod1(j, g.Ny), 1] - 2*f.data[i, j, 1] + f.data[i, decmod1(j, g.Ny), 1]) / g.Δy
        ∇²f.data[i, j, end] = (f.data[i, j, end-1] - f.data[i, j, end]) / g.Δz +
                              (f.data[incmod1(i, g.Nx), j, end] - 2*f.data[i, j, end] + f.data[decmod1(i, g.Nx), j, end]) / g.Δx +
                              (f.data[i, incmod1(j, g.Ny), end] - 2*f.data[i, j, end] + f.data[i, decmod1(j, g.Ny), end]) / g.Δy
    end
    nothing
end

∇²! (generic function with 1 method)

In [44]:
N = (20, 20, 20)
L = (20, 20, 20)
g = RegularCartesianGrid(N, L; dim=3, FloatType=Float64)

3-dimensional (Float64) regular Cartesian grid
(Nx, Ny, Nz) = (20, 20, 20)
(Lx, Ly, Lz) = (20.0, 20.0, 20.0)
(Δx, Δy, Δz) = (1.0, 1.0, 1.0)

In [45]:
f = CellField(g)
f.data .= rand(Float64, size(g));

In [51]:
∇²f1 = ∇²(f.data);

∇²f2 = CellField(g)
∇²!(g, f, ∇²f2)

In [52]:
∇²f1 ≈ ∇²f2.data

true

In [48]:
@show size(∇²f2.data)
∇²f2.data[:, :, 5]

size(∇²f2.data) = (20, 20, 20)


20×20 Array{Float64,2}:
  3.3306    -2.63313   -0.110992  …  -0.416087     1.13412   -0.538915 
 -2.0142     3.22163   -1.81413      -1.29002     -2.03925    1.31161  
  3.05538   -1.4272     2.51859      -1.08699      0.411961  -0.542666 
 -0.199     -2.26904   -2.78481       3.70391      1.50845   -1.80936  
  0.593972   1.48618   -1.36133      -3.81807      3.16588   -1.78918  
  2.78286   -1.76541   -0.75179   …   1.57958     -3.0622    -1.77697  
  0.331314   2.14532    0.263278      0.917325    -0.37308    0.485756 
  0.81724    0.770389   1.40957       2.48664     -3.21593    1.16199  
 -2.51754    1.1348    -0.814165     -1.84673      1.13592   -0.0504461
 -1.88864    3.09626   -1.47269       2.01498     -1.65599   -0.332634 
  0.399403  -2.61253   -0.521811  …   0.00888251   1.62487   -1.54581  
 -0.816565   1.09564    0.973311     -3.15032      2.48666   -1.01016  
  1.60292   -0.226699  -0.355699     -0.111442    -2.5403    -0.290688 
  2.9774    -1.90558    0.014861     -2.

In [49]:
@show size(∇²f3.data)
∇²f3.data[:, :, 5]

size(∇²f3.data) = (20, 20, 20)


20×20 Array{Float64,2}:
  3.3306    -2.63313   -0.110992  …  -0.416087     1.13412   -0.538915 
 -2.0142     3.22163   -1.81413      -1.29002     -2.03925    1.31161  
  3.05538   -1.4272     2.51859      -1.08699      0.411961  -0.542666 
 -0.199     -2.26904   -2.78481       3.70391      1.50845   -1.80936  
  0.593972   1.48618   -1.36133      -3.81807      3.16588   -1.78918  
  2.78286   -1.76541   -0.75179   …   1.57958     -3.0622    -1.77697  
  0.331314   2.14532    0.263278      0.917325    -0.37308    0.485756 
  0.81724    0.770389   1.40957       2.48664     -3.21593    1.16199  
 -2.51754    1.1348    -0.814165     -1.84673      1.13592   -0.0504461
 -1.88864    3.09626   -1.47269       2.01498     -1.65599   -0.332634 
  0.399403  -2.61253   -0.521811  …   0.00888251   1.62487   -1.54581  
 -0.816565   1.09564    0.973311     -3.15032      2.48666   -1.01016  
  1.60292   -0.226699  -0.355699     -0.111442    -2.5403    -0.290688 
  2.9774    -1.90558    0.014861     -2.

In [50]:
@show minimum(∇²f1), minimum(∇²f2.data);
@show maximum(∇²f1), maximum(∇²f2.data);
@show mean(∇²f1), mean(∇²f2.data);
@show sum(∇²f1), sum(∇²f2.data);

(minimum(∇²f1), minimum(∇²f2.data)) = (-5.0926428783880615, -5.0926428783880615)
(maximum(∇²f1), maximum(∇²f2.data)) = (4.80425313661758, 4.80425313661758)
(mean(∇²f1), mean(∇²f2.data)) = (-7.327471962526034e-18, -4.773959005888173e-18)
(sum(∇²f1), sum(∇²f2.data)) = (-5.861977570020827e-14, -3.8191672047105385e-14)


In [35]:
x = 4 + 5

9

In [36]:
y = 4 + 5
  + 1 + 1

2

In [39]:
@show y;

y = 9


In [40]:
z = 4 + 5 +
    1 + 1

11

In [42]:
@show z;

z = 11
